In [4]:
import os
import pandas as pd
import pandas as pd
import numpy as np
import re
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, chi2
from gensim.models import Word2Vec
from sklearn.preprocessing import normalize
from string import punctuation
# keep it in mind, pip install py-readability-metrics, otherwise this file is null
from readability import Readability
# import readability
import language_tool_python

from preprocessing import tokenize
pd.options.display.max_columns = 250
pd.options.display.max_colwidth = 160

import features as util
from raw_utils import save_to_csv
from preprocessing import dataset_add_columns

from ast import literal_eval

cwd = os.getcwd()
csv_path = os.path.join(cwd, 'data/csv/')

generic_spam_tokens = 'generic_spam_tokens.csv'
non_targeted_tokens = 'non_targeted_tokens.csv'

generic_spam_tokens = pd.read_csv(os.path.join(csv_path, generic_spam_tokens), index_col=0,
                                  converters={'body': literal_eval})
non_targeted_tokens = pd.read_csv(os.path.join(csv_path, non_targeted_tokens), index_col=0,
                                       converters={'body': literal_eval})


In [5]:
text_col = generic_spam_tokens['body']
text_combined = text_col.map(' '.join)

In [6]:
text_col

0        [gracia, happi, hear, improv, ceh, phillip, love, pm, carri, hollomon, hou, ect, ect, susan, harrison, hou, ect, ect, cc, subject, elizabeth, soto, want, fo...
1        [literatur, cognit, scienc, editor, seek, submiss, collect, aim, facilit, discus, disciplin, literari, critic, cognit, scienc, center, read, literatur, spir...
2        [u, geolog, survey, nation, earthquak, inform, center, world, data, center, seismolog, denver, repli, emailaddress, emailaddress, follow, releas, unit, stat...
3        [start, date, hourahead, hour, ancillari, schedul, award, varianc, detect, log, messag, par, file, portland, westdesk, california, schedul, iso, final, sche...
4        [sound, like, good, trip, space, avail, think, dive, captain, check, avail, cours, dive, captain, also, hesit, swallow, whole, whale, shark, tell, hous, rem...
                                                                                      ...                                                                  

In [7]:
text_combined

0        gracia happi hear improv ceh phillip love pm carri hollomon hou ect ect susan harrison hou ect ect cc subject elizabeth soto want follow guy convers recent ...
1        literatur cognit scienc editor seek submiss collect aim facilit discus disciplin literari critic cognit scienc center read literatur spirit cool reason subs...
2        u geolog survey nation earthquak inform center world data center seismolog denver repli emailaddress emailaddress follow releas unit state geolog survey nat...
3                        start date hourahead hour ancillari schedul award varianc detect log messag par file portland westdesk california schedul iso final schedul txt
4        sound like good trip space avail think dive captain check avail cours dive captain also hesit swallow whole whale shark tell hous remodel would probabl bles...
                                                                                      ...                                                                  

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 3))
tfidf = tfidf_vectorizer.fit_transform(text_combined)


In [9]:
tfidf_score = tfidf_vectorizer.fit_transform(text_combined).toarray()
tfidf_terms = pd.DataFrame(tfidf_score, columns=tfidf_vectorizer.get_feature_names_out())

In [10]:
# 设置NMF的参数
n_components = 10  # 这是你想要的主题数量
init = 'nndsvd'  # 使用NNDSVD作为初始化方法
max_iter = 200

# 创建NMF实例并拟合数据
nmf = NMF(n_components=n_components, init=init, max_iter=max_iter)
W = nmf.fit_transform(tfidf_terms)  # 文档-主题矩阵
H = nmf.components_  # 主题-术语矩阵
H = pd.DataFrame(H, columns=tfidf_vectorizer.get_feature_names_out())
W_normalized = normalize(W, norm='l1', axis=1)

In [11]:
tfidf_terms

,abl,abstract,accept,access,account,action,activ,ad,addit,address,agre,agreement,align,allow,alreadi,also,america,american,amount,analysi,announc,anoth,applic,approv,april,area,around,ask,asset,assist,associ,attach,august,author,avail,back,bank,base,begin,believ,best,big,bill,billion,board,book,br,bring,build,busi,buy,ca,california,call,capac,capit,case,cash,cc,cc subject,center,chang,charg,check,chri,citi,class,click,close,co,come,comment,commiss,committe,commun,compani,complet,comput,concern,confer,confidenti,confirm,consid,contact,contain,continu,contract,control,copi,corp,corp enron,corp enron enron,corpor,cost,could,creat,credit,current,custom,daili,data,databas,date,davi,david,day,de,deal,decemb,demand,depart,detail,develop,differ,direct,director,discus,distribut,doc,document,draft,due,earn,ect,ect cc,...,respons,result,return,review,richard,right,risk,robert,rule,run,sale,san,save,say,schedul,scott,second,section,secur,see,sell,send,send monday,send tuesday,septemb,servic,session,set,sever,share,short,show,sign,sinc,site,size,smith,soon,speak,special,specif,st,standard,start,state,steve,still,stock,street,structur,studi,subject,submit,suggest,suppli,support,sure,susan,system,tabl,take,talk,td,td td,team,technolog,tell,term,texa,text,th,thank,thing,think,three,thursday,time,today,top,total,tr,trade,transact,transmiss,travel,tri,tuesday,two,type,uk,understand,unit,univers,updat,urladdress,urladdress urladdress,use,user,util,valu,version,view,visit,volum,want,way,web,wednesday,week,well,whether,width,william,within,without,word,work,workshop,world,would,would like,write,www,year,york
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.096189,0.123603,0.000000,0.000000,0.0,0.146774,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.143887,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.504380,0.146586,...,0.140412,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.109883,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.077628,0.0,0.0,0.0,0.000000,0.000000,0.171724,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.08655,0.0,0.119062,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.147415,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.117516,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.096817,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.164941,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.148957,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.157341,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.126487,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.130334,0.0,0.000000,0.0,0.126913,0.0,0.00000,0.000000,0.0

In [12]:
W

array([[1.84426492e-02, 1.55291705e-01, 0.00000000e+00, ...,
        8.07854595e-04, 1.02430075e-02, 0.00000000e+00],
       [4.73686011e-03, 0.00000000e+00, 0.00000000e+00, ...,
        4.77955974e-03, 0.00000000e+00, 3.55804712e-03],
       [0.00000000e+00, 0.00000000e+00, 3.52332791e-02, ...,
        1.75340279e-03, 1.29252556e-02, 5.00316152e-02],
       ...,
       [3.72504966e-02, 9.13000398e-02, 4.97598860e-05, ...,
        3.90604356e-03, 1.67230951e-03, 2.77268116e-03],
       [2.12789825e-02, 5.86235605e-04, 0.00000000e+00, ...,
        2.91513624e-02, 2.13926173e-02, 2.35361138e-02],
       [5.52110315e-03, 1.40242500e-03, 0.00000000e+00, ...,
        1.49638727e-03, 5.83349618e-03, 3.19701710e-02]])

In [13]:
H

,abl,abstract,accept,access,account,action,activ,ad,addit,address,agre,agreement,align,allow,alreadi,also,america,american,amount,analysi,announc,anoth,applic,approv,april,area,around,ask,asset,assist,associ,attach,august,author,avail,back,bank,base,begin,believ,best,big,bill,billion,board,book,br,bring,build,busi,buy,ca,california,call,capac,capit,case,cash,cc,cc subject,center,chang,charg,check,chri,citi,class,click,close,co,come,comment,commiss,committe,commun,compani,complet,comput,concern,confer,confidenti,confirm,consid,contact,contain,continu,contract,control,copi,corp,corp enron,corp enron enron,corpor,cost,could,creat,credit,current,custom,daili,data,databas,date,davi,david,day,de,deal,decemb,demand,depart,detail,develop,differ,direct,director,discus,distribut,doc,document,draft,due,earn,ect,ect cc,...,respons,result,return,review,richard,right,risk,robert,rule,run,sale,san,save,say,schedul,scott,second,section,secur,see,sell,send,send monday,send tuesday,septemb,servic,session,set,sever,share,short,show,sign,sinc,site,size,smith,soon,speak,special,specif,st,standard,start,state,steve,still,stock,street,structur,studi,subject,submit,suggest,suppli,support,sure,susan,system,tabl,take,talk,td,td td,team,technolog,tell,term,texa,text,th,thank,thing,think,three,thursday,time,today,top,total,tr,trade,transact,transmiss,travel,tri,tuesday,two,type,uk,understand,unit,univers,updat,urladdress,urladdress urladdress,use,user,util,valu,version,view,visit,volum,want,way,web,wednesday,week,well,whether,width,william,within,without,word,work,workshop,world,would,would like,write,www,year,york
0,0.239821,0.000000,0.014955,0.034420,0.052340,0.035039,0.058438,0.060938,0.074918,0.054386,0.146527,0.000000,0.000000,0.085352,0.173606,0.458065,0.000000,0.042781,0.056832,0.012024,0.000000,0.218477,0.000000,0.000000,0.035651,0.052651,0.292553,0.398079,0.000799,0.108348,0.051912,0.000000,0.029328,0.000000,0.127139,0.616351,0.026463,0.042376,0.120447,0.221612,0.447815,0.217782,0.096511,0.000000,0.063805,0.224227,0.001209,0.179333,0.084839,0.239161,0.105448,0.031542,0.000000,0.839450,0.000000,0.031209,0.200022,0.077685,0.182960,0.192038,0.000000,0.275775,0.038833,0.286935,0.180369,0.069953,0.109740,0.000000,0.116039,0.003109,0.600952,0.069817,0.000000,0.019383,0.005345,0.000000,0.083188,0.000000,0.128012,0.038052,0.000000,0.110543,0.089501,0.014798,0.000000,0.118791,0.024410,0.046615,0.000000,0.000000,0.000000,0.000000,0.000000,0.071764,0.615705,0.038486,0.000000,0.107477,0.000000,0.000000,0.026078,0.013175,0.090495,0.000000,0.052708,0.697869,0.000000,0.479397,0.023505,0.000000,0.000000,0.181999,0.013557,0.158334,0.065652,0.034396,0.253452,0.000000,0.000000,0.000000,0.026095,0.081155,0.038326,0.000000,0.075968,...,0.104525,0.037031,0.111390,0.031765,0.054807,0.336519,0.071445,0.030553,0.025852,0.269760,0.015119,0.053207,0.073300,0.536715,0.00000,0.110827,0.083557,0.000000,0.008553,0.549796,0.016617,0.113589,0.000000,0.000000,0.037145,0.000000,0.002953,0.237578,0.114901,0.066808,0.110742,0.182861,0.139231,0.265116,0.045652,0.028717,0.000000,0.309961,0.215725,0.045040,0.059658,0.114542,0.139689,0.219653,0.000000,0.164198,0.467623,0.000000,0.000000,0.014811,0.000000,0.123449,0.000000,0.197175,0.000000,0.106717,0.446738,0.108642,0.018532,0.023334,0.721082,0.640048,0.004807,0.000000,0.365266,0.000000,0.438323,0.072820,0.000000,0.000000,0.312115,1.081626,0.492381,1.129382,0.097340,0.154985,1.059216,0.488298,0.071386,0.064150,0.000000,0.000000,0.000000,0.000000,0.120098,0.508454,0.101688,0.274712,0.000000,0.000000,0.182845,0.017484,0.00000,0.176185,0.000000,0.00000,0.272780,0.000000,0.000000,0.052848,0.004864,0.005396,0.111835,0.033333,1.079941,0.390672,0.000000,0.140159,0.805890,0.597754,0.126753,0.000000,0.018039,0.049473,0.106714,0.168351,0.937359,0.000000,0.067865,1.447279,0.628180,0.182158,0.000000,0.392998,0.000000
1,0.000000,0.000000,0.000000,0.011365,0.009522,0.000000,0.000000,0.007883,0.000000,0.000000,0.020701,0.096438,0.000000,0.000000,0.00

In [14]:
top_terms = util.get_topics(tfidf_vectorizer, H)

In [15]:
top_terms

[['know',
  'get',
  'go',
  'would',
  'let',
  'let know',
  'like',
  'think',
  'thank',
  'want'],
 ['ect',
  'hou',
  'hou ect',
  'ect ect',
  'hou ect ect',
  'ect cc',
  'ect ect cc',
  'cc',
  'cc subject',
  'pm'],
 ['emailaddress',
  'emailaddress emailaddress',
  'emailaddress emailaddress emailaddress',
  'emailaddress cc',
  'emailaddress subject',
  'mailto',
  'mailto emailaddress',
  'emailaddress send',
  'cc',
  'mailto emailaddress send'],
 ['languag',
  'univers',
  'linguist',
  'paper',
  'de',
  'confer',
  'abstract',
  'edu',
  'english',
  'workshop'],
 ['urladdress',
  'click',
  'urladdress urladdress',
  'free',
  'email',
  'report',
  'avail',
  'publish',
  'view',
  'mail'],
 ['enron',
  'enron enron',
  'corp',
  'na',
  'na enron',
  'na enron enron',
  'corp enron',
  'corp enron enron',
  'enron develop',
  'ee'],
 ['schedul',
  'date',
  'hour',
  'final',
  'iso',
  'start',
  'file',
  'california',
  'type',
  'id'],
 ['origin messag',
  'mess